In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("yelp_review.csv")

In [4]:
data.describe()

,stars,useful,funny,cool
count,5.261668e+06,5.261668e+06,5.261668e+06,5.261668e+06
mean,3.727739e+00,1.385085e+00,5.091961e-01,5.860917e-01
std,1.433593e+00,4.528727e+00,2.686168e+00,2.233706e+00
min,1.000000e+00,-1.000000e+00,0.000000e+00,-1.000000e+00
25%,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,5.000000e+00,2.000000e+00,0.000000e+00,1.000000e+00
max,5.000000e+00,3.364000e+03,1.481000e+03,1.105000e+03


In [4]:
data.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,Super simple place but amazing nonetheless. It...,0,0,0
1,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,Small unassuming place that changes their menu...,0,0,0
2,MV3CcKScW05u5LVfF6ok0g,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,2016-05-28,Lester's is located in a beautiful neighborhoo...,0,0,0
3,IXvOzsEMYtiJI0CARmj77Q,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,2016-05-28,Love coming here. Yes the place always needs t...,0,0,0
4,L_9BTb55X0GDtThi6GlZ6w,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q,4,2016-05-28,Had their chocolate almond croissant and it wa...,0,0,0


In [3]:
data = data[['stars', 'text']]

In [6]:
data.head()

,stars,text
0,5,Super simple place but amazing nonetheless. It...
1,5,Small unassuming place that changes their menu...
2,5,Lester's is located in a beautiful neighborhoo...
3,4,Love coming here. Yes the place always needs t...
4,4,Had their chocolate almond croissant and it wa...


In [ ]:
from WordProcessing import dictionary_make
import multiprocessing as mp
from multiprocessing.dummy import Pool
import sys
from collections import Counter

print('Making new columns')
data['length'] = pd.Series(index=data.index)
data['adjectives'] = pd.Series(index=data.index)
data['propernouns'] = pd.Series(index=data.index)
data['nouns'] = pd.Series(index=data.index)
data['adverbs'] = pd.Series(index=data.index)
data['verbs'] = pd.Series(index=data.index)

n_jobs = 4
inputs = np.array_split(data, n_jobs)
tasks = []
j = 0
out = mp.Queue()
widgets = mp.Queue()
processes = []
results = []

for i in inputs:
    processes.append(mp.Process(target=dictionary_make, args=(i,out,widgets,j)))
    j += 1
for i in range(n_jobs):
    processes[i].start()
    print("Started " + str(i))
    if (((i + 1) % 2) == 0):
        for j in range(i -1, i + 1):
            results.append(out.get())
        for j in range(i -1, i + 1):
            print("Making " + str(j) + " join")
            processes[j].join()
#for p in processes:
    #p.start()
bars = [widgets.get() for p in processes]
for f in bars:
    print(f)
    
#results = [out.get() for p in processes]
#print("Joining Processes")
#for p in processes:
    #p.join()
frames = []
total_count = Counter()
for result in results:
    total_count += result[1]
    frames.append(result[0])
    
new_data = pd.concat(frames) 
  
print()
print('Number of words to query: ' + str(len(total_count)))
print()
sys.stdout.flush()
print(total_count.most_common(50))
print()
sys.stdout.flush()
new_data.head()

Making new columns
Started 0
Started 1


In [ ]:
# keep tokens with a min occurrence
min_occurance = 6
vocab = [k for k,c in dictionary.items() if c >= min_occurance]
print('Number of words to query reduced to: ' + str(len(vocab)))

#Save dictionary
tosave = '\n'.join(vocab)
# open file
file = open('vocab3.txt', 'w')
# write text
file.write(tosave)
# close file
file.close()

In [ ]:
with open("dictionary.txt", "w+") as f:
    for k,v in  dictionary.most_common():
        f.write( "{} {}\n".format(k,v) )

In [ ]:
new_data.to_csv("text_data.csv")

In [ ]:
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

def load_vocab():
    # load the vocabulary
    vocab_filename = 'vocab3.txt'
    vocab2 = load_doc(vocab_filename)
    vocab2 = vocab2.split()
    return set(vocab2)

vocab = load_vocab()

In [7]:
from ipywidgets import FloatProgress
from IPython.display import display
import time

def refresh_data_(data):
    data = data.clone()
    max_count = len(data)
    #progress bar
    f = FloatProgress(min=0, max=max_count) # instantiate the bar
    display(f) # display the bar
    data['length'] = pd.Series(index=data.index)
    data['adjectives'] = pd.Series(index=data.index)
    data['propernouns'] = pd.Series(index=data.index)
    data['nouns'] = pd.Series(index=data.index)
    data['adverbs'] = pd.Series(index=data.index)
    data['verbs'] = pd.Series(index=data.index)
    for index, row in data.iterrows():
        tokens, length, adjectives, adverbs, nouns, propernouns, verbs = clean(row['text'], tag=True)
        tokens = [w for w in tokens if w in vocab]
        row['text'] = ' '.join(tokens)
        row['length'] = length
        row['adjectives'] = adjectives
        row['adverbs'] = adverbs
        row['nouns'] = nouns
        row['propernouns'] = propernouns
        row['verbs'] = verbs
        f.value += 1 # signal to increment the progress bar
    
    return data

In [ ]:
from multiprocessing.dummy import Pool as ThreadPool
def refresh_data(data, n_jobs = 4):
    pool = ThreadPool(n_jobs) 
    results = pool.map(refresh_data_, data)
    
    return pd.DataFrame(results)        